In [2]:
import os.path as osp
from math import ceil

import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import DenseGraphConv, GCNConv, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch

path = osp.join(osp.dirname(osp.realpath('C:\\Users\\ivano\\Desktop\\DSMMaster\\CryptoThesis\\data')), '..', 'data', 'PROTEINS')
dataset = TUDataset(path, name='PROTEINS').shuffle()
average_nodes = int(dataset.data.x.size(0) / len(dataset))
n = (len(dataset) + 9) // 10
test_dataset = dataset[:n]
val_dataset = dataset[n:2 * n]
train_dataset = dataset[2 * n:]
test_loader = DataLoader(test_dataset, batch_size=20)
val_loader = DataLoader(val_dataset, batch_size=20)
train_loader = DataLoader(train_dataset, batch_size=20)

Extracting C:\Users\ivano\Desktop\DSMMaster\data\PROTEINS\PROTEINS\PROTEINS.zip
Processing...
Done!


In [9]:
print(average_nodes)

39


In [10]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=32):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        num_nodes = ceil(0.5 * average_nodes)
        self.pool1 = Linear(hidden_channels, num_nodes)

        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        num_nodes = ceil(0.5 * num_nodes)
        self.pool2 = Linear(hidden_channels, num_nodes)

        self.conv3 = DenseGraphConv(hidden_channels, hidden_channels)

        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()

        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch)

        s = self.pool1(x)
        x, adj, mc1, o1 = dense_mincut_pool(x, adj, s, mask)

        x = self.conv2(x, adj).relu()
        s = self.pool2(x)

        x, adj, mc2, o2 = dense_mincut_pool(x, adj, s)

        x = self.conv3(x, adj)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), mc1 + mc2, o1 + o2


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)

In [11]:
def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, mc_loss, o_loss = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(out, data.y.view(-1)) + mc_loss + o_loss
        loss.backward()
        loss_all += data.y.size(0) * float(loss)
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all = 0

    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        loss_all += data.y.size(0) * float(loss)
        correct += int(pred.max(dim=1)[1].eq(data.y.view(-1)).sum())

    return loss_all / len(loader.dataset), correct / len(loader.dataset)


best_val_acc = test_acc = 0
best_val_loss = float('inf')
patience = start_patience = 50
for epoch in range(1, 150):
    train_loss = train(epoch)
    _, train_acc = test(train_loader)
    val_loss, val_acc = test(val_loader)
    if val_loss < best_val_loss:
        test_loss, test_acc = test(test_loader)
        best_val_acc = val_acc
        patience = start_patience
    else:
        patience -= 1
        if patience == 0:
            break
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, '
          f'Train Acc: {train_acc:.3f}, Val Loss: {val_loss:.3f}, '
          f'Val Acc: {val_acc:.3f}, Test Loss: {test_loss:.3f}, '
          f'Test Acc: {test_acc:.3f}')

Epoch: 001, Train Loss: 1.058, Train Acc: 0.601, Val Loss: 1.062, Val Acc: 0.580, Test Loss: 1.074, Test Acc: 0.571
Epoch: 002, Train Loss: 1.010, Train Acc: 0.729, Val Loss: 1.048, Val Acc: 0.670, Test Loss: 1.061, Test Acc: 0.661
Epoch: 003, Train Loss: 0.983, Train Acc: 0.750, Val Loss: 1.035, Val Acc: 0.679, Test Loss: 1.050, Test Acc: 0.679
Epoch: 004, Train Loss: 0.964, Train Acc: 0.750, Val Loss: 1.027, Val Acc: 0.670, Test Loss: 1.049, Test Acc: 0.688
Epoch: 005, Train Loss: 0.952, Train Acc: 0.757, Val Loss: 1.020, Val Acc: 0.670, Test Loss: 1.043, Test Acc: 0.688
Epoch: 006, Train Loss: 0.941, Train Acc: 0.760, Val Loss: 1.013, Val Acc: 0.670, Test Loss: 1.039, Test Acc: 0.679
Epoch: 007, Train Loss: 0.931, Train Acc: 0.759, Val Loss: 1.009, Val Acc: 0.661, Test Loss: 1.037, Test Acc: 0.688
Epoch: 008, Train Loss: 0.923, Train Acc: 0.768, Val Loss: 1.004, Val Acc: 0.688, Test Loss: 1.037, Test Acc: 0.696
Epoch: 009, Train Loss: 0.916, Train Acc: 0.773, Val Loss: 0.999, Val Ac

KeyboardInterrupt: 